In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

In [ ]:
# Path to your ChromeDriver executable
chrome_driver_path = r"C:\Users\Syndictech\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no browser window)
chrome_options.add_argument('--disable-blink-features=AutomationControlled')

# Set up the WebDriver
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)


In [ ]:

# User input for search query
search_query = input("Enter the name to search: ")
apply_website_filter = input("Do you want to filter by a specific website (yes/no)? ").strip().lower()

# Determine website filter if needed
if apply_website_filter == 'yes':
    website_filter = input("Enter the website domain to filter (e.g., website.com): ").strip()
    search_query += ":" + website_filter
    print("Modified search query:", search_query)
else:
    website_filter = None

def apply_filter(href):
    # Exclude links from Google internal paths and empty hrefs
    return href and not href.startswith('/search') and 'google.com' not in href

def extract_links(page_number):
    links = driver.find_elements(By.XPATH, '//a[@href]')
    for link in links:
        href = link.get_attribute('href')
        if apply_filter(href):
            print(href)
    print(f"\nPage {page_number} links extracted.\n")

# Perform the initial search
driver.get('https://www.google.com/')
search_box = driver.find_element(By.NAME, 'q')
search_box.send_keys(search_query)
search_box.send_keys(Keys.RETURN)

# Wait for results to load
time.sleep(3)

# Page number counter
page_number = 1

# Extract links from the first page
extract_links(page_number)

# Navigate through the pages
while True:
    try:
        # Check for the presence of the "Next" button
        next_button = driver.find_elements(By.ID, 'pnnext')
        if next_button:
            next_button[0].click()
        else:
            print("No more 'Next' button found.")
            break

        # Increment the page number
        page_number += 1

        # Wait for the next page to load
        time.sleep(6)

        # Extract links from the next page
        extract_links(page_number)
    except Exception as e:
        print("An error occurred or no more pages:", e)
        break

# Close the browser
driver.quit()